In [1]:
import face_recognition
import cv2
import os
from datetime import datetime

# フォルダ準備
known_dir = "known"
save_dir = "detected_faces"
os.makedirs(known_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# 顔データ読み込み
known_face_encodings = []
known_face_names = []

print("🧠 顔画像の登録中...")
for filename in os.listdir(known_dir):
    if filename.endswith((".jpg", ".png")):
        path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            known_face_encodings.append(encoding[0])
            known_face_names.append(os.path.splitext(filename)[0])
            print(f"✅ {filename} 登録成功")
        else:
            print(f"⚠️ {filename} に顔が検出されませんでした")

if not known_face_encodings:
    print("❌ 登録された顔画像がありません。終了します。")
    exit()

# Webカメラ起動
video_capture = cv2.VideoCapture(0)
print("📸 Webカメラ起動中（qで終了）")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ 映像取得失敗")
        break

    # BGR→RGB変換
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 顔検出と特徴抽出
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        if known_face_encodings:
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            if len(face_distances) > 0:
                best_match_index = face_distances.argmin()
                if face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.65)[best_match_index]:
                    name = known_face_names[best_match_index]

        # 顔画像の切り出し
        face_image = frame[top:bottom, left:right]
        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{name}_{now}.jpg"
        save_path = os.path.join(save_dir, filename)
        cv2.imwrite(save_path, face_image)
        print(f"💾 保存：{save_path}")

        # 枠とラベル表示
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 0), 2)

    # 表示
    cv2.imshow("Face Auto Save", frame)

    # 終了キー
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 終了します")
        break

video_capture.release()
cv2.destroyAllWindows()

🧠 顔画像の登録中...
✅ ishiba_shigeru.jpg 登録成功
✅ yamada.jpg 登録成功
✅ trump.jpg 登録成功
📸 Webカメラ起動中（qで終了）
💾 保存：detected_faces/yamada_20250506_135009.jpg
💾 保存：detected_faces/yamada_20250506_135010.jpg
💾 保存：detected_faces/yamada_20250506_135010.jpg
💾 保存：detected_faces/yamada_20250506_135010.jpg
💾 保存：detected_faces/yamada_20250506_135011.jpg
💾 保存：detected_faces/yamada_20250506_135011.jpg
💾 保存：detected_faces/yamada_20250506_135012.jpg
💾 保存：detected_faces/yamada_20250506_135012.jpg
💾 保存：detected_faces/yamada_20250506_135012.jpg
💾 保存：detected_faces/yamada_20250506_135013.jpg
💾 保存：detected_faces/yamada_20250506_135013.jpg
💾 保存：detected_faces/yamada_20250506_135013.jpg
💾 保存：detected_faces/yamada_20250506_135014.jpg
💾 保存：detected_faces/yamada_20250506_135014.jpg
💾 保存：detected_faces/yamada_20250506_135014.jpg
💾 保存：detected_faces/yamada_20250506_135015.jpg
💾 保存：detected_faces/yamada_20250506_135015.jpg
💾 保存：detected_faces/yamada_20250506_135016.jpg
💾 保存：detected_faces/yamada_20250506_135016.jpg
💾 保存：detected_f

# 補足：保存頻度の制御（同じ人を連続で保存しない）

In [2]:
import face_recognition
import cv2
import os
from datetime import datetime
from time import time

# フォルダ準備
known_dir = "known"
save_dir = "detected_faces"
os.makedirs(known_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# 登録顔の読み込み
known_face_encodings = []
known_face_names = []

print("🧠 顔画像の登録中...")
for filename in os.listdir(known_dir):
    if filename.endswith((".jpg", ".png")):
        path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            known_face_encodings.append(encoding[0])
            known_face_names.append(os.path.splitext(filename)[0])
            print(f"✅ {filename} 登録成功")
        else:
            print(f"⚠️ {filename} に顔が検出されませんでした")

if not known_face_encodings:
    print("❌ 登録された顔画像がありません。終了します。")
    exit()

# 顔ごとの直近保存時間記録
last_saved_time = {}
save_interval_sec = 5  # 同じ人物の保存間隔（秒）

# Webカメラ起動
video_capture = cv2.VideoCapture(0)
print("📸 Webカメラ起動中（qで終了）")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ 映像取得失敗")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    current_time = time()

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        if known_face_encodings:
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            if len(face_distances) > 0:
                best_match_index = face_distances.argmin()
                if face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.65)[best_match_index]:
                    name = known_face_names[best_match_index]

        # 保存間隔チェック
        last_time = last_saved_time.get(name, 0)
        if current_time - last_time > save_interval_sec:
            face_image = frame[top:bottom, left:right]
            now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{name}_{now_str}.jpg"
            save_path = os.path.join(save_dir, filename)
            cv2.imwrite(save_path, face_image)
            print(f"💾 保存：{save_path}")
            last_saved_time[name] = current_time
        else:
            print(f"⏸️ スキップ：{name}（{int(current_time - last_time)}秒以内）")

        # 表示
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 0), 2)

    cv2.imshow("Face Auto Save (with interval)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 終了します")
        break

video_capture.release()
cv2.destroyAllWindows()

🧠 顔画像の登録中...
✅ ishiba_shigeru.jpg 登録成功
✅ yamada.jpg 登録成功
✅ trump.jpg 登録成功
📸 Webカメラ起動中（qで終了）
💾 保存：detected_faces/yamada_20250506_135833.jpg
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存：detected_faces/yamada_20250506_135838.jpg
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
🛑 終了します


# 保存される顔画像に名前とスコアを直接書き込む

In [1]:
import face_recognition
import cv2
import os
from datetime import datetime
from time import time

# フォルダ準備
known_dir = "known"
save_dir = "detected_faces"
os.makedirs(known_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

# 顔データ登録
known_face_encodings = []
known_face_names = []

print("🧠 顔画像の登録中...")
for filename in os.listdir(known_dir):
    if filename.endswith((".jpg", ".png")):
        path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(path)
        encoding = face_recognition.face_encodings(image)
        if encoding:
            known_face_encodings.append(encoding[0])
            known_face_names.append(os.path.splitext(filename)[0])
            print(f"✅ {filename} 登録成功")
        else:
            print(f"⚠️ {filename} に顔が検出されませんでした")

if not known_face_encodings:
    print("❌ 登録された顔画像がありません。終了します。")
    exit()

# 保存制御（人物ごと）
last_saved_time = {}
save_interval_sec = 5

# Webカメラ起動
video_capture = cv2.VideoCapture(0)
print("📸 Webカメラ起動中（qで終了）")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ 映像取得失敗")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    current_time = time()

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        confidence = 0.0

        if known_face_encodings:
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            if len(face_distances) > 0:
                best_match_index = face_distances.argmin()
                confidence = 1 - face_distances[best_match_index]
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.65)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

        label_text = f"{name} ({confidence:.2f})"

        # 保存制御（5秒以内はスキップ）
        last_time = last_saved_time.get(name, 0)
        if current_time - last_time > save_interval_sec:
            face_image = frame[top:bottom, left:right].copy()

            # 名前とスコアを顔画像に描画
            cv2.putText(face_image, label_text, (5, 25), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 0), 2)

            now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{name}_{now_str}.jpg"
            save_path = os.path.join(save_dir, filename)
            cv2.imwrite(save_path, face_image)
            print(f"💾 保存（ラベル付き）：{save_path}")
            last_saved_time[name] = current_time
        else:
            print(f"⏸️ スキップ：{name}（{int(current_time - last_time)}秒以内）")

        # 表示画面には通常通り描画
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, label_text, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 0), 2)

    cv2.imshow("Face Save with Label", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 終了します")
        break

video_capture.release()
cv2.destroyAllWindows()

🧠 顔画像の登録中...
✅ ishiba_shigeru.jpg 登録成功
✅ yamada.jpg 登録成功
✅ trump.jpg 登録成功
📸 Webカメラ起動中（qで終了）
💾 保存（ラベル付き）：detected_faces/yamada_20250506_141242.jpg
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存（ラベル付き）：detected_faces/yamada_20250506_141247.jpg
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（3秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
⏸️ スキップ：yamada（4秒以内）
💾 保存（ラベル付き）：detected_faces/yamada_20250506_141253.jpg
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（0秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（1秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yamada（2秒以内）
⏸️ スキップ：yam